<a href="https://colab.research.google.com/github/varunhari2020/CourseProject/blob/main/CausalTopicModeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
!pip install pyLDAvis

     |████████████████████████████████| 1.6MB 5.7MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97712 sha256=ec1fe1fecb490141c5e93e9d7311390f615354f6dea6cc9402ee8e8f67b2187d
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
Successfully built pyLDAvis


In [4]:
import re
import numpy as np
import numpy.linalg as la
import pandas as pd
from pprint import pprint
import datetime

# NLTK
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk import ngrams

# Gensim
import gensim
from gensim import models
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models import Phrases # TODO: to create bigrams with

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt 

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
stop_words = stopwords.words('english')
stop_words.extend(['mr', 'ms', 'said'])

def tokenize(content, lemmatize=False):
    words = gensim.utils.simple_preprocess(content, deacc=True)  # tokenizes
    return words

def remove_stopwords(content):
    words = []
    for word in content:
        if word in stop_words:
            continue
        words.append(word)
    return words

rows = []
dates = []
articles = []
for month in range(5, 11):
    with open("/content/drive/Data/NYTimes/"+ str(month) + ".txt") as f:
        for i, line in enumerate(f):
            date, article = line.split(",", 1)
            timestamp = datetime.datetime.strptime(date, "%Y-%m-%dT%H:%M:%S%z")
            tokenized = tokenize(article)
            destopped = remove_stopwords(tokenized)

            articles.append(destopped)
            dates.append(timestamp)
            rows.append([timestamp, destopped])

df = pd.DataFrame(rows, columns=["date", "content"]) 
unique_dates = sorted(list(set(df["date"])))
print (unique_dates)
df

FileNotFoundError: ignored

In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(articles)

# Attempt at filtering out words that appear too frequently
# id2word.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)
# id2word.filter_extremes(no_above=0.5)


# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in articles]


tfidf = models.TfidfModel(corpus)
corpus = tfidf[corpus]


# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('ago', 0.07712049873418031),
  ('awesome', 0.23220574510227418),
  ('backup', 0.2198823985449398),
  ('backups', 0.2515408271170864),
  ('bases', 0.19264069440348208),
  ('bellinger', 0.27548950382241366),
  ('bench', 0.1896343958919212),
  ('bush', 0.007894722475376273),
  ('came', 0.08612993720379283),
  ('catcher', 0.26148042600790294),
  ('clay', 0.2135830725972484),
  ('games', 0.1562902360625982),
  ('girardi', 0.27548950382241366),
  ('homer', 0.21658937110880933),
  ('jim', 0.1245222966630543),
  ('joe', 0.1146922085996351),
  ('leyritz', 0.27548950382241366),
  ('speed', 0.17969479700110466),
  ('stole', 0.20587332073042908),
  ('strength', 0.13402729061444735),
  ('turner', 0.2108175460504276),
  ('two', 0.04788545375938528),
  ('versatility', 0.27548950382241366),
  ('whose', 0.0887458288821732),
  ('yankee', 0.20825706839694694),
  ('yankees', 0.19264069440348208),
  ('years', 0.05159983565074285)]]

In [ ]:
k = 10

# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=k, 
                                           minimum_phi_value=0.5, # min threshold for word probabilities
                                           passes=5,
                                           alpha='auto',  # assuming that topic distribution is assymetric. Not all topics equally represented in corpus.
                                           update_every=1,
                                           per_word_topics=True)

# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=articles, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -10.335340102501334

Coherence Score:  0.498118082888875


In [ ]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.281978 -0.010030       1        1  80.539883
6     -0.072979 -0.123439       2        1   6.816635
7     -0.071312  0.056962       3        1   5.016989
2     -0.035634  0.040698       4        1   2.465846
0     -0.023432  0.009694       5        1   1.330145
3     -0.020399  0.008006       6        1   1.093652
9     -0.017158  0.006529       7        1   0.872190
4     -0.016352  0.005475       8        1   0.729956
5     -0.013075  0.003395       9        1   0.610154
8     -0.011636  0.002710      10        1   0.524549, topic_info=             Term       Freq      Total Category  logprob  loglift
27724         var  30.000000  30.000000  Default  30.0000  30.0000
6754       string  29.000000  29.000000  Default  29.0000  29.0000
3598         else  27.000000  27.000000  Default  28.0000  28.0000
1034        nader  61.000000  61.000000  Default  27.0000  27.0000
1045        ralph  25.000000  25.000000  Default  26.0000  26.0000
...           ...        ...        ...      ...      ...      ...
11677    tutorial   0.074280   0.602824  Topic10  -8.1246   3.1566
14107  multimedia   0.082599   0.885003  Topic10  -8.0184   2.8788
17983    piercing   0.073357   0.632680  Topic10  -8.1371   3.0958
9327     matthews   0.066843   0.614469  Topic10  -8.2301   3.0320
6540       juries   0.065523   0.932724  Topic10  -8.2500   2.5947

[444 rows x 6 columns], token_table=       Topic      Freq          Term
term                                
507        1  1.003403      abortion
9193       3  0.805918        abroad
27         3  1.054138    accomplish
28         2  0.330957  accomplished
28         3  0.661913  accomplished
...      ...       ...           ...
8540       4  1.114611      warheads
11643      1  0.680240      wherever
78         1  0.993349         would
1068       1  0.961873          york
1068       4  0.026719          york

[266 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 7, 8, 3, 1, 4, 10, 5, 6, 9])

In [ ]:
for l in lda_model.get_document_topics(corpus)[:10]:
    print (l)

[(1, 0.6156847), (2, 0.01631999), (4, 0.20373307), (6, 0.06801649), (7, 0.028621836), (8, 0.0344656)]
[(1, 0.763563), (2, 0.013859917), (6, 0.033660006), (7, 0.088353015), (9, 0.068763435)]
[(1, 0.79859394), (2, 0.013141152), (6, 0.08126964), (7, 0.037291188), (9, 0.049772587)]
[(1, 0.87178653), (6, 0.029328), (7, 0.067593485)]
[(1, 0.8022314), (2, 0.010433715), (6, 0.14003201), (7, 0.01831812)]
[(1, 0.77750266), (2, 0.101369575), (6, 0.057995867), (7, 0.024414107)]
[(1, 0.84217304), (6, 0.027844958), (7, 0.06244437), (9, 0.039514713)]
[(1, 0.7187414), (2, 0.011190535), (6, 0.03845834), (7, 0.20053901)]
[(1, 0.82175153), (2, 0.058945935), (6, 0.064610116), (7, 0.022253001)]
[(0, 0.0109442), (1, 0.730404), (2, 0.018160004), (3, 0.01012647), (6, 0.169174), (7, 0.031808387)]


In [ ]:
# for any given day, you look at all the diff topics and identify the prob of that topic
date_topic_prob = np.zeros((len(unique_dates), k))
for date, article in zip(df["date"], lda_model.get_document_topics(corpus)):
  i = unique_dates.index(date)
  for topic, prob in article:
    date_topic_prob[i][topic] += prob 
date_topic_prob = date_topic_prob/date_topic_prob.max(axis=0)

In [ ]:
date_topic_prob

array([[0.00960165, 0.12697915, 0.01179038, ..., 0.20422308, 0.21812294,
        0.0933737 ],
       [0.01475436, 0.09657251, 0.02128822, ..., 0.07538966, 0.        ,
        0.26194383],
       [0.40982177, 0.32274864, 0.11762854, ..., 0.52242613, 0.10376247,
        0.        ],
       ...,
       [0.10482325, 0.33340277, 0.35393836, ..., 0.28412118, 0.03938651,
        0.10970976],
       [0.40246888, 0.95823687, 1.        , ..., 0.71573004, 0.        ,
        0.489494  ],
       [0.10647875, 0.49076954, 0.55658635, ..., 0.4426987 , 0.        ,
        0.31024785]])

In [ ]:
# Example of norming by col
# d = np.array([
#     [1000, 10, 0.5],
#     [765, 5, 0.35],
#     [800, 7, 0.09], ])
# d/d.max(axis=0)

array([[1.   , 1.   , 1.   ],
       [0.765, 0.5  , 0.7  ],
       [0.8  , 0.7  , 0.18 ]])

In [ ]:
t0 = unique_dates[4]
t0

Timestamp('2000-05-05 05:00:00+0000', tz='UTC')